# Análisis de Componentes Principales Crimen Estados Unidos

En este notebook vamos a realizar el Análisis de Componentes Principales del dataset de crimen en Estados Unidos. Intentaremos graficar los datos para sacar algunas conclusiones junto el análisis cluster que se realizará en un notebook posterior.

<div>
<img src="./media/crime.jpeg" width="500"/>
</div>

Para ello empleamos los siguientes módulos:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Comenzamos cargando los datos:

In [ ]:
crimen_data = pd.read_csv('./data/USArrests.csv', index_col = 0)

Comprobamos que los datos se hayan cargado de manera adecuada:

In [ ]:
crimen_data.head()

In [ ]:
crimen_data.describe()

In [ ]:
crimen_data.shape

Observando el dataframe podemos ver que la escala de las variables es distinta en cada una por lo que será necesario escalarlas para poder calcular las componentes principales. Procedemos al escalado:

In [ ]:
from sklearn.preprocessing import scale
X_crime = pd.DataFrame(scale(crimen_data), index=crimen_data.index, columns=crimen_data.columns)

In [ ]:
X_crime.head()

El cálculo de los vectores de carga se encuentra automatizado por Scikit-Learn, basta importar el módulo necesario:

In [ ]:
from sklearn.decomposition import PCA
pca_loadings = pd.DataFrame(PCA().fit(X_crime).components_.T, index=X_crime.columns, columns=['V1', 'V2', 'V3', 'V4'])
pca_loadings

También podemos representar cada uno de los estados en las coordenadas generadas por las nuevas componentes:

In [ ]:
pca = PCA()
df_plot = pd.DataFrame(pca.fit_transform(X_crime), columns=['PC1', 'PC2', 'PC3', 'PC4'], index=X_crime.index)
df_plot.head()

En base a estas coordenadas y tomando solo las dos primeras componentes principales podemos visualizar los datos en un diagrama de dispersión en el que representamos también los vectores de carga:

In [ ]:
fig , ax1 = plt.subplots(figsize=(20,10))
ax1.set_xlim(-3.5,3.5)
ax1.set_ylim(-3.5,3.5)
# Represnetar las dos primeras componentes principales
for i in df_plot.index:
    ax1.annotate(i, (-df_plot.PC1.loc[i], -df_plot.PC2.loc[i]), ha='center')
# Dibujar las líneas de referencia
ax1.hlines(0,-3.5,3.5, linestyles='dotted', colors='grey')
ax1.vlines(0,-3.5,3.5, linestyles='dotted', colors='grey')
ax1.set_xlabel('Primera Componente Principal')
ax1.set_ylabel('Segunda Componente Principal')
# Construimos un segundo par de ejes para los vectores de carga
ax2 = ax1.twinx().twiny()
ax2.set_ylim(-1,1)
ax2.set_xlim(-1,1)
ax2.set_xlabel('En rojo: Vectores de carga de las componentes principales', color='red')
# Colocamos las etiquetas de los vectores de carga
a = 1.07
for i in pca_loadings[['V1', 'V2']].index:
    ax2.annotate(i, (-pca_loadings.V1.loc[i]*a, -pca_loadings.V2.loc[i]*a), color='red')
# Representamos los vectores de cargas
ax2.arrow(0,0,-pca_loadings.V1[0], -pca_loadings.V2[0])
ax2.arrow(0,0,-pca_loadings.V1[1], -pca_loadings.V2[1])
ax2.arrow(0,0,-pca_loadings.V1[2], -pca_loadings.V2[2])
ax2.arrow(0,0,-pca_loadings.V1[3], -pca_loadings.V2[3]);

### Hipótesis

A partir de la representación de los datos sobre las dos primeras componentes principales podemos sacar las siguientes conclusiones:

* La primera componente principal recoge principalmente información de las variables asalto, violación y asesinato. Digamos que esta primera componente estudia principalmente la criminalidad en los distintos estados.

* La segunda componente principal sin embargo se ve principalmente influenciada por la variable UrbanPop mientras que las variables asociadas al crimen pierden importancia. Se podría decir que esta segunda componente principal expresa la urbanización de los estados (recordemos que UrbanPop es el porcentaje de población que vive en áreas urbanas).

* Las variables de crimen se encuentran próximas y apuntan en la misma dirección, esto quiere decirnos que están correlacionadas (algo que ya vimos en el análisis exploratorio previo). La variable UrbanPop se encuentra bastante separada lo cual indica una menor correlación aunque sigue apuntando en la misma dirección por lo que la correlación seguirá siendo directa.

* Cabe suponer que los estados  que se encuentran próximos en la representación tendrán indices similares de criminalidad y urbanización.

Estas son las principales hipótesis que podemos plantear simplemente visualizando la representación de los datos sobre las dos primeras componentes principales. Para evaluar la validez de estas hipótesis debemos comprobar cuánta varianza explican estas dos primeras componentes principales.

Podemos observar a partir del modelo cuánta varianza explica cada componente:

In [ ]:
pca.explained_variance_

Si preferimos el ratio de varianza (suele ser más sencillo e indicativo) también está disponible:

In [ ]:
pca.explained_variance_ratio_

También podemos calcular la proporción de varianza acumulada:

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

Construimos ambos gráficos de sedimentación:

In [ ]:
plt.figure(figsize=(7,5))
plt.plot([1,2,3,4], pca.explained_variance_ratio_, '-o')
plt.ylabel('Proporción de varianza explicada')
plt.xlabel('Componente Principal')
plt.xlim(0.75,4.25)
plt.ylim(0,1.05)
plt.xticks([1,2,3,4]);

In [ ]:
plt.figure(figsize=(7,5))
plt.plot([1,2,3,4], np.cumsum(pca.explained_variance_ratio_), '-s')
plt.ylabel('Proporción de varianza explicada acumulada')
plt.xlabel('Componente Principal')
plt.xlim(0.75,4.25)
plt.ylim(0,1.05)
plt.xticks([1,2,3,4]);

Observamos que con solo dos componentes estamos expresando el 86% de la información del dataset por lo que las conclusiones sacadas de nuestro estudio de la gráfica son altamente fiables. Además observamos que el número ideal de componentes principales a calcular será 2 o 3 dependiendo de la cantidad de información que nos propongamos expresar.